In [1]:
import os, io, shutil
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm

import numpy as np
from tensorboardX import SummaryWriter
from dataloader.handhygiene import HandHygiene
from spatial_transforms import (
    Compose, Normalize, Scale, CenterCrop, CornerCrop, MultiScaleCornerCrop,
    MultiScaleRandomCrop, RandomHorizontalFlip, ToTensor)
from temporal_transforms import (
    MirrorLoopPadding, LoopPadding, TemporalRandomCrop, 
    TemporalCenterCrop, TemporalRandomChoice)
from openpose_transforms import CropTorso, MultiScaleTorsoRandomCrop

/usr/local/lib/python3.5/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
model_name = 'i3d'
img_size = 224
crop_size = img_size
batch_size = 4
clip_len = 64
num_classes = 1

torch.manual_seed(100)
data_name = 'anesthesia'
dataset_path = os.path.join(os.getcwd(), 'data')

In [3]:
scales = 1.0
sample_duration = 64
sample_size = 224
mean=[110.63666788, 103.16065604, 96.29023126]
std=[38.7568578, 37.88248729, 40.02898126]

scales=np.linspace(1, 1.75, num=4)
openpose_transform = MultiScaleTorsoRandomCrop(scales, sample_size)
spatial_transform = Compose([
            Scale(sample_size),
            CenterCrop(sample_size),
            RandomHorizontalFlip(),
            ToTensor(1), 
            Normalize(mean, std)])

temporal_transform = TemporalRandomChoice([
            LoopPadding(sample_duration),
            MirrorLoopPadding(sample_duration),
            TemporalRandomCrop(sample_duration),
            TemporalCenterCrop(sample_duration)])

In [4]:
dataset = {
    'train':HandHygiene(dataset_path, split='train', clip_len=clip_len, 
                        spatial_transform=spatial_transform,
                        openpose_transform=openpose_transform,
                        temporal_transform=temporal_transform, num_workers=16),
    'val':HandHygiene(dataset_path, split='val', clip_len=clip_len, 
                        spatial_transform=spatial_transform,
                        openpose_transform=openpose_transform,
                        temporal_transform=temporal_transform, num_workers=16)}

dataloaders = {
    'train': DataLoader(dataset['train'], batch_size=batch_size, shuffle=True, num_workers=16),
    'val': DataLoader(dataset['val'], batch_size=batch_size, shuffle=False, num_workers=16)}

Number of /data/private/minjee-video/handhygiene/data/images/train videos: 91
Number of /data/private/minjee-video/handhygiene/data/images/val videos: 8


In [5]:
from train import get_models
from train import train

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
i3d_rgb, i3d_flow = get_models(num_classes, True)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    i3d_rgb = nn.DataParallel(i3d_rgb)
    i3d_flow = nn.DataParallel(i3d_flow)
i3d_rgb.to(device)
i3d_flow.to(device)

criterion = F.binary_cross_entropy
optims={'rgb': None, 'flow':None}
feature_extract=True
            
def trainable_params(model, mode='rgb'):
    params_to_update = model.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    optims[mode] = optim.SGD(model.parameters(), lr=1e-6, momentum=0.9, weight_decay=1e-7)
    
trainable_params(i3d_rgb, 'rgb')
trainable_params(i3d_flow, 'flow')

Let's use 4 GPUs!


In [8]:
train((i3d_rgb, i3d_flow), dataloaders, optims, criterion, device, num_epochs=50)

  0%|          | 0/50 [00:00<?, ?it/s]

loop padding
mirror loop padding
temporal random crop
mirror loop padding
temporal random crop
mirror loop padding
temporal random crop
temporal random crop
temporal center crop
mirror loop padding
mirror loop padding
temporal center crop
temporal random crop
temporal center crop
mirror loop padding
loop padding
1.75
1.75
1.25
1.0
1.75
1.75
1.75
1.75
1.5
1.5
1.75
1.5
1.5
1.0
1.75
1.5
temporal random crop
loop padding
temporal center crop
mirror loop padding
temporal center crop
temporal random crop
mirror loop padding
temporal center crop
mirror loop padding
loop padding
loop padding
temporal random crop
mirror loop padding
mirror loop padding
temporal center crop
mirror loop padding
1.75
1.0
1.25
1.75
1.5
1.25
1.25
1.25
1.0
1.25
1.0
1.75
1.75
1.5
1.25
1.25
mirror loop padding
loop padding
temporal center crop
temporal random crop
temporal center crop
loop padding
temporal center crop
loop padding
mirror loop padding
mirror loop padding
loop padding
temporal center crop
temporal random

1.0
1.0
out_sigmoid:tensor([[0.5312],
        [0.3656],
        [0.4216],
        [0.2289]], device='cuda:0')
1.25
temporal center crop
temporal center crop
out_sigmoid:tensor([[0.2840],
        [0.4217],
        [0.5373],
        [0.2103]], device='cuda:0')
temporal center crop
1.0
out_sigmoid:tensor([[0.0644],
        [0.2471],
        [0.2018],
        [0.2349]], device='cuda:0')
1.75
out_sigmoid:tensor([[0.4812],
        [0.6469],
        [0.2112],
        [0.1376]], device='cuda:0')
1.0
1.5
out_sigmoid:tensor([[0.5041],
        [0.0696],
        [0.7293],
        [0.0387]], device='cuda:0')
out_sigmoid:tensor([[0.3201],
        [0.6890],
        [0.8137],
        [0.7579]], device='cuda:0')
1.75
temporal random crop
out_sigmoid:tensor([[0.1303],
        [0.4768],
        [0.3668],
        [0.2114]], device='cuda:0')
1.0
1.5
out_sigmoid:tensor([[0.6204],
        [0.1775],
        [0.5952],
        [0.1591]], device='cuda:0')
temporal random crop
loop padding
out_sigmoid:tensor([[0.

NameError: name 'dataloaders' is not defined